In [12]:
import cv2
import os
import mediapipe as mp
import pandas as pd
from lib.utilities import *
from lib.joint_angles import JointAngle
from lib.body_parts import BodyPart

In [13]:
n = 8
angles = [0] * n
bp = BodyPart()

In [14]:
mpPose = mp.solutions.pose
video = '../assets/sample-video/sample-video01.mp4'
cap = cv2.VideoCapture(video)

In [15]:
data = {
    'Angles': [],
    'Phase': [],
    'Correction': []
}

In [16]:
output_folder = '../assets/sample-frames/sample-video01/'
os.makedirs(output_folder, exist_ok=True)
frameCnt = 0

with mpPose.Pose(min_detection_confidence=0.75, min_tracking_confidence=0.75) as pose:
    while cap.isOpened():
        ret, image = cap.read()

        frameName = f'frame_{frameCnt}.jpg'
        framePath = os.path.join(output_folder,frameName)
        cv2.imwrite(framePath, image)
        frameCnt += 1

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image.flags.writeable = True
        
        try:
            lm = results.pose_landmarks.landmark
            jointAngle = JointAngle(lm)
            bp.cords = detect_joints(lm)
            angles = jointAngle.body_angles(bp)
            data['Angles'].append(angles)
            data['Phase'].append(None)
            data['Correction'].append('Widen your legs')
        except:
            pass

error: OpenCV(4.6.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'


In [11]:
df = pd.DataFrame(data)
df
df.to_csv('../dataset/standard/barbellCurl.csv')

In [ ]:
df

In [ ]:
PhaseName = {
    'Null': 0,
    'Down': 1,
    'Up': 2
}

CorrectionName = {
    'Good': 0,
    'Move up - Left Arm': 1,
    'Move up - Right Arm': 2,
    'Move down - Left Arm': 3,
    'Move down - Right Arm': 4,
    'Move left - Left Arm': 5,
    'Move left - Right Arm': 6,
    'Move right - Left Arm': 7,
    'Move right - Right Arm': 8,
    # 'Widen your legs': 9
}

In [ ]:

fileName = '../dataset/angles.csv'
df = pd.read_csv(fileName)

for i in range(len(df['Phase'])):
    df['Phase'][i] = PhaseName[df['Phase'][i]]

In [ ]:
df = df.drop('Unnamed: 0', axis=1)
df.to_csv('../dataset/angles_final.csv')